In [1]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()



[nltk_data] Error loading punkt: <urlopen error [WinError 10061] No
[nltk_data]     connection could be made because the target machine
[nltk_data]     actively refused it>


In [6]:
# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
import random
import json

In [7]:
# import our chat-bot intents file
import pandas as pd
intents=pd.read_json('intents.json')

In [8]:
intents

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
3,"{'tag': 'chatbot', 'patterns': ['Who built thi..."
4,"{'tag': 'location', 'patterns': ['What is your..."
5,"{'tag': 'connect', 'patterns': ['Give me your ..."
6,"{'tag': 'movies', 'patterns': ['Which is your ..."
7,"{'tag': 'about', 'patterns': ['Who are you?', ..."


In [9]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [10]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
52 unique stemmed words ["'s", 'about', 'account', 'address', 'ani', 'anyon', 'are', 'built', 'bye', 'can', 'chatbot', 'connect', 'day', 'favourit', 'give', 'good', 'goodby', 'hello', 'help', 'hi', 'how', 'i', 'is', 'later', 'link', 'locat', 'me', 'media', 'movi', 'name', 'out', 'reach', 'recommend', 'see', 'social', 'some', 'suggest', 'tell', 'thank', 'that', 'there', 'thi', 'to', 'way', 'we', 'what', 'where', 'which', 'who', 'you', 'your', 'yourself']


In [11]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training, dtype=object)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [12]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 

In [13]:
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=8, verbose=1)
model.save("model.pkl")

Epoch 1/1000
4/4 [==============================] - 1s 11ms/step - loss: 2.1554 - accuracy: 0.0370
Epoch 2/1000
4/4 [==============================] - 0s 8ms/step - loss: 2.1324 - accuracy: 0.0370
Epoch 3/1000
4/4 [==============================] - 0s 9ms/step - loss: 2.1128 - accuracy: 0.0370
Epoch 4/1000
4/4 [==============================] - 0s 7ms/step - loss: 2.0947 - accuracy: 0.0370
Epoch 5/1000
4/4 [==============================] - 0s 5ms/step - loss: 2.0764 - accuracy: 0.0741
Epoch 6/1000
4/4 [==============================] - 0s 5ms/step - loss: 2.0590 - accuracy: 0.0741
Epoch 7/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.0421 - accuracy: 0.1111
Epoch 8/1000
4/4 [==============================] - 0s 3ms/step - loss: 2.0253 - accuracy: 0.1111
Epoch 9/1000
4/4 [==============================] - 0s 5ms/step - loss: 2.0078 - accuracy: 0.1111
Epoch 10/1000
4/4 [==============================] - 0s 5ms/step - loss: 1.9924 - accuracy: 0.1111
Epoch 11/1000
4/4 

INFO:tensorflow:Assets written to: model.pkl\assets


In [14]:
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )

In [15]:
from keras.models import load_model
model = load_model("model.pkl")

In [16]:
# restoring all the data structures
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']

In [17]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [18]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    bag=np.array(bag)
    return(bag)

In [19]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    bag = bow(sentence, words)
    results = model.predict(np.array([bag]))
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results[0]) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [20]:
response("Who are you?")

1/1 [==============================] - 0s 353ms/step
Thanks for asking. I am Mayank Bajaj, coder by profession but ML enthusiast by passion.


In [21]:
response("Where are you from?")

1/1 [==============================] - 0s 100ms/step
You can visit India to meet us
